In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
"""
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
"""
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

"\nfor dirname, _, filenames in os.walk('/kaggle/input'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))\n"

In [2]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:3000"

In [3]:
!pip install monai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.7/990.7 kB 3.7 MB/s eta 0:00:00


In [4]:
!pip install GPUtil

from GPUtil import showUtilization as gpu_usage

  Preparing metadata (setup.py) ... - done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7411 sha256=3aaa46f163a184e257ab11bbf38c22a9670f76a7b7d55aafd57ff6c84ca9fe5b
  Stored in directory: /root/.cache/pip/wheels/6e/f8/83/534c52482d6da64622ddbf72cd93c35d2ef2881b78fd08ff0c
Successfully built GPUtil


In [5]:
! pip install pylibjpeg pylibjpeg-libjpeg pydicom
! pip install -U python-gdcm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 36.6 MB/s eta 0:00:00


In [6]:
import pylibjpeg
import libjpeg

In [7]:
from pydicom.pixel_data_handlers.util import apply_voi_lut
from torch.utils.data import Dataset, DataLoader

In [8]:
import monai.transforms as transforms

In [9]:
import cv2
import matplotlib.pyplot as plt
import pydicom as dicom
import torch
import torchvision as vision
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GroupKFold, KFold
from torch.cuda.amp import GradScaler
from torch.cuda.amp import autocast
from torchvision.models import feature_extraction
from tqdm.notebook import tqdm
import pydicom as dicom
import gc

In [10]:
TRAIN_CSV_PATH = '../input/rsna-2022-cervical-spine-fracture-detection/train.csv'
TRAIN_DATA_PATH = '../input/cervical-spine-pt/train_volumes'

In [11]:
device ='cuda' if torch.cuda.is_available() else 'cpu'
batch_size = 8

In [12]:
device

'cuda'

In [13]:
"""
def load_data(path):
    img = dicom.dcmread(path)
    data = apply_voi_lut(img.pixel_array, img)
    data = cv2.resize(data, (img_size, img_size), interpolation=cv2.INTER_NEAREST)
    norm = (np.min(data), np.max(data))
    if norm[1] != 0:
        data = (data-norm[0]) / norm[1]
    data=(data * 255).astype(np.uint8)
    return cv2.cvtColor(data, cv2.COLOR_GRAY2RGB)
"""

'\ndef load_data(path):\n    img = dicom.dcmread(path)\n    data = apply_voi_lut(img.pixel_array, img)\n    data = cv2.resize(data, (img_size, img_size), interpolation=cv2.INTER_NEAREST)\n    norm = (np.min(data), np.max(data))\n    if norm[1] != 0:\n        data = (data-norm[0]) / norm[1]\n    data=(data * 255).astype(np.uint8)\n    return cv2.cvtColor(data, cv2.COLOR_GRAY2RGB)\n'

In [14]:
"""img = load_data('../input/rsna-2022-cervical-spine-fracture-detection/train_images/1.2.826.0.1.3680043.6200/1.dcm')
plt.figure()
plt.imshow(img)
plt.title('Image 0')

img = load_data(f'{TRAIN_DATA_PATH}/1.2.826.0.1.3680043.10005/1.dcm')
plt.figure()
plt.imshow(img)
plt.title('Image 1')"""

"img = load_data('../input/rsna-2022-cervical-spine-fracture-detection/train_images/1.2.826.0.1.3680043.6200/1.dcm')\nplt.figure()\nplt.imshow(img)\nplt.title('Image 0')\n\nimg = load_data(f'{TRAIN_DATA_PATH}/1.2.826.0.1.3680043.10005/1.dcm')\nplt.figure()\nplt.imshow(img)\nplt.title('Image 1')"

In [15]:
train_df = pd.read_csv(TRAIN_CSV_PATH)
train_df.head(5)

,StudyInstanceUID,patient_overall,C1,C2,C3,C4,C5,C6,C7
0,1.2.826.0.1.3680043.6200,1,1,1,0,0,0,0,0
1,1.2.826.0.1.3680043.27262,1,0,1,0,0,0,0,0
2,1.2.826.0.1.3680043.21561,1,0,1,0,0,0,0,0
3,1.2.826.0.1.3680043.12351,0,0,0,0,0,0,0,0
4,1.2.826.0.1.3680043.1363,1,0,0,0,0,1,0,0


In [16]:
def create_label_string(row):
    label = str(row['patient_overall'])
    for i in range(7):
        label += str(row['C'+str(i+1)])
    label_int = int(label, 2)
    return label_int

In [17]:
train_df['labels'] = train_df.apply(lambda x: create_label_string(x), axis = 1)

In [18]:
train_df.head(3)

,StudyInstanceUID,patient_overall,C1,C2,C3,C4,C5,C6,C7,labels
0,1.2.826.0.1.3680043.6200,1,1,1,0,0,0,0,0,224
1,1.2.826.0.1.3680043.27262,1,0,1,0,0,0,0,0,160
2,1.2.826.0.1.3680043.21561,1,0,1,0,0,0,0,0,160


In [19]:
class FractureDetectDataset(Dataset):
    def __init__(self, df, path, transform=None):
        super().__init__()
        self.df = df
        self.path = path
        self.transform = transform
    
    def __getitem__(self, i):
        PATH = os.path.join(self.path, self.df.iloc[i, 0])
        try:
            img = torch.load(f'{PATH}.pt')
            img_tensor = img.permute((2, 0, 1))
        except Exception as e:
            print(e.message)
            pass
        #print(temp.shape)
        if self.transform is not None:
            inputs = self.transform(img_tensor)
        else:
            inputs = img_tensor
        targets = torch.as_tensor(self.df.iloc[i,[1, 2, 3, 4, 5, 6 , 7, 8]].astype('float32').values)
        return {"inputs": inputs.unsqueeze(0), "target": targets}

    def __len__(self):
        return len(self.df)

In [20]:
"""train_transform = transforms.Compose([transforms.Resize((img_size, img_size, stack_size)),
                                      transforms.ToTensor()
])"""

'train_transform = transforms.Compose([transforms.Resize((img_size, img_size, stack_size)),\n                                      transforms.ToTensor()\n])'

In [21]:
PATH = os.path.join(TRAIN_DATA_PATH, train_df.iloc[0, 0])
try:
    img = torch.load(f'{PATH}.pt')
    img_tensor = img.permute((2, 0, 1))
except Exception as e:
    print(e)
    pass
targets = torch.as_tensor(train_df.iloc[0,[1, 2, 3, 4, 5, 6 , 7, 8]].astype('float32').values)
print(targets, img_tensor.shape)

tensor([1., 1., 1., 0., 0., 0., 0., 0.]) torch.Size([40, 100, 100])


In [22]:
gpu_usage()

| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |


In [23]:
from sklearn.model_selection import train_test_split

In [24]:
list1 = list(train_df['labels'].value_counts())
count = 0
for val in list1:
    if val<2:
        count +=1
print(count)

30


In [25]:
counts = train_df['labels'].value_counts()

res = train_df[~train_df['labels'].isin(counts[counts < 2].index)]
print(res.shape, train_df.shape)

(1989, 10) (2019, 10)


In [26]:
train_df_edited = res

In [27]:
df_train, df_valid = train_test_split(train_df_edited, test_size=0.2, random_state=43, stratify = train_df_edited.labels)
df_train.shape, df_valid.shape

((1591, 10), (398, 10))

In [28]:
train_dataset = FractureDetectDataset(df = df_train, path = TRAIN_DATA_PATH, transform=None)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [29]:
gpu_usage()

| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |


In [30]:
import time

In [31]:
start = time.time()
data = next(iter(train_dataloader))
print(data['inputs'].shape, data['target'].shape, "\n", time.time()-start)

torch.Size([8, 1, 40, 100, 100]) torch.Size([8, 8]) 
 0.12045884132385254


In [32]:
data['target']

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 1., 0.],
        [1., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [33]:
import torch.nn as nn

In [34]:
class Detector(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv3d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=0) #(-1, 1, 40, 100, 100) -> #(-1, 16, 38, 98, 98)
        self.pooling1 = nn.MaxPool3d(kernel_size=2, stride=2, padding=0) #(8, 16, 38, 98,98) -> (8, 16, 19, 49, 49)
        self.norm1 = nn.BatchNorm3d(num_features=16) #(8, 16, 19, 49, 49)
        self.layer1 = nn.Sequential(self.conv1, self.pooling1, self.norm1) #(8, 3, 40, 100, 100) -> (8, 16, 19, 49, 49)
        
        self.conv2 = nn.Conv3d(in_channels=16, out_channels=32, kernel_size=3, stride=(1, 2, 2), padding=0) #(8, 16, 19, 49, 49) -> (8, 32, 17, 24, 24)
        #self.norm2 = nn.BatchNorm3d(num_features = 64)
    
        self.layer2 = nn.Sequential(self.conv2)
        
        #self.conv4 = nn.Conv3d(in_channels=64, out_channels=256, kernel_size=3, stride=1, padding=0) #(8, 32, 17, 47, 47) -> (8, 256, 15, 45, 45)
        self.final_pool = nn.AvgPool3d((7, 5, 5), stride = (2, 3, 3)) #(8, 32, 17, 24, 24) -> (8, 32, 6, 7, 7)
        #self.norm4 = nn.BatchNorm3d(num_features = 256)
        self.layer3 = nn.Sequential( self.final_pool)
        
        self.flat = nn.Flatten()
        
        self.relu = nn.ReLU()
        self.layer4 = nn.Sequential(nn.Linear(in_features=32*6*7*7, out_features=256), self.relu,
                                    nn.Linear(in_features=256, out_features=32), self.relu,
                                    nn.Linear(in_features=32, out_features = 8)
                                    )
        
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        #print(x.shape)
        out = self.layer1(x)
        #print(out.shape)
        out = self.layer2(out)
        #print(out.shape)
        out = self.layer3(out)
        #print(out.shape)
        
        output = self.flat(out)
        output = self.layer4(output)
        #print(output.shape)
        return output
    
    def predict(self, x):
        preds = self.forward(x)
        preds = self.sigmoid(preds)
        return preds

In [35]:
device

'cuda'

In [36]:
model = Detector()
model.to(device)

Detector(
  (conv1): Conv3d(1, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (pooling1): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (norm1): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): Conv3d(1, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (1): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv2): Conv3d(16, 32, kernel_size=(3, 3, 3), stride=(1, 2, 2))
  (layer2): Sequential(
    (0): Conv3d(16, 32, kernel_size=(3, 3, 3), stride=(1, 2, 2))
  )
  (final_pool): AvgPool3d(kernel_size=(7, 5, 5), stride=(2, 3, 3), padding=0)
  (layer3): Sequential(
    (0): AvgPool3d(kernel_size=(7, 5, 5), stride=(2, 3, 3), padding=0)
  )
  (flat): Flatten(start_dim=1, end_dim=-1)
  (relu): ReLU()
  (layer4): Sequential(
    (0): Linear(in_features=9408, out_features=256, bias=T

In [37]:
gpu_usage()

| ID | GPU | MEM |
------------------
|  0 |  2% |  4% |


In [38]:
start = time.time()
trial_tensor = torch.rand(8, 1, 40, 100, 100).to(device)
print(trial_tensor.get_device(), "\n", model.predict(trial_tensor), time.time()-start)

0 
 tensor([[0.5108, 0.4658, 0.5227, 0.4628, 0.4758, 0.4756, 0.4886, 0.5363],
        [0.5113, 0.4668, 0.5218, 0.4620, 0.4753, 0.4749, 0.4874, 0.5354],
        [0.5102, 0.4649, 0.5222, 0.4618, 0.4735, 0.4761, 0.4878, 0.5378],
        [0.5105, 0.4643, 0.5218, 0.4623, 0.4741, 0.4762, 0.4891, 0.5379],
        [0.5108, 0.4667, 0.5232, 0.4628, 0.4751, 0.4752, 0.4883, 0.5366],
        [0.5101, 0.4659, 0.5210, 0.4623, 0.4745, 0.4752, 0.4885, 0.5385],
        [0.5111, 0.4641, 0.5229, 0.4619, 0.4742, 0.4758, 0.4880, 0.5375],
        [0.5109, 0.4655, 0.5237, 0.4621, 0.4748, 0.4762, 0.4887, 0.5373]],
       device='cuda:0', grad_fn=<SigmoidBackward0>) 6.214350700378418


In [39]:
torch.cuda.empty_cache()
gc.collect()

103

In [40]:
print(len(train_dataloader))

199


In [41]:
import copy

In [42]:
def train(epoch, model, opt, lr_scheduler, criterion, dataloader, device):
    print(f"Epoch: {epoch}")
    model.train()
    
    running_loss = 0
    total_loss = []
    lrs = []
    total_size = 0
    correct = 0
    
    start = time.time()
    for i, data in enumerate(dataloader):
        #gpu_usage()
        gc.collect()
        inputs = data["inputs"].to(device)
        targets = data["target"].to(device)
        #print(i)
        inputs = inputs.type(torch.cuda.FloatTensor)
        targets = targets.type(torch.cuda.FloatTensor)
        #print(ids.shape, "ids")
        batch_size = inputs.size(0)
        #gpu_usage()
        output = model.forward(inputs)
        gc.collect()
        del inputs
        loss = criterion(output, targets)
        #print(output.shape, targets.shape, loss.shape)
        loss.mean().backward()
        opt.step()
        opt.zero_grad()
        if i%4 == 0 or i+1==len(dataloader):
            lr_scheduler.step()
        total_loss.append(torch.sum(loss))
        gc.collect()
        del loss
        #gpu_usage()
        torch.cuda.empty_cache()
        output = torch.sigmoid(output)
        predictions = torch.as_tensor((output - 0.6) > 0, dtype=torch.int32)
        correct += (predictions == targets).float().sum().item()
        gc.collect()
        #gpu_usage()
        del predictions
        del targets
        del output
        #gpu_usage()
        torch.cuda.empty_cache()
        #print(predictions, "\n", targets, "\n", correct)
        total_size += batch_size
        accuracy = correct/(total_size*8)
        #print(correct, total_size)
        lrs.append(opt.param_groups[0]['lr'])
        if i%40==0:
            print(time.time()-start, ":time")
            print("Training Loss over a batch: {:.4f}; Training Loss over a single value: {:.4f} Accuracy: {:.2f}%".format(torch.mean(torch.stack(total_loss)), (torch.mean(torch.stack(total_loss)))/(batch_size*8), accuracy*100))
            start = time.time()
        break
    return torch.mean(torch.stack(total_loss)), accuracy, lrs

def valid_eval(model, dataloader, device):
    model.eval()
    
    total_size = 0
    total_loss = []
    correct = 0
    with torch.no_grad():
        for data in dataloader:
            gc.collect()
            torch.cuda.empty_cache()
            inputs = data["inputs"].to(device)
            targets = data["target"].to(device)

            inputs = inputs.type(torch.cuda.FloatTensor)
            targets = targets.type(torch.cuda.FloatTensor)
            #print(ids.shape, "ids")
            batch_size = inputs.size(0)

            output = model.forward(inputs)
            gc.collect()
            del inputs
            loss = criterion(output, targets)
            total_loss.append(torch.sum(loss))
            gc.collect()
            del loss
            torch.cuda.empty_cache()
            output = torch.sigmoid(output)

            predictions = torch.as_tensor((output - 0.6) > 0, dtype=torch.int32)
            correct += (predictions == targets).float().sum().item()
            gc.collect()
            del predictions
            del targets
            del output
            torch.cuda.empty_cache()
            total_size += batch_size
            #gpu_usage()
        accuracy = correct/(total_size*8)
    
    print("Validation Loss over a batch: {:.4f}; Training Loss over a single value: {:.4f} Accuracy: {:.2f}%".format(torch.mean(torch.stack(total_loss)),(torch.mean(torch.stack(total_loss)))/(batch_size*8), accuracy*100))
    model.train()
    return torch.mean(torch.stack(total_loss)), accuracy

In [43]:
def train_epochs(model, opt, lr_scheduler, criterion, device, epochs=25, mode='trial'):
    min_avg_loss = 1e9
    max_accuracy = 0
    history = {"Train Loss": [], "Valid Loss": [], "Train Acc": [], "Valid Acc": [], "LRs": []}
    
    for epoch in range(epochs):
        train_loss, train_accuracy, lrs = train(epoch, model, opt, lr_scheduler, criterion, dataloader = train_dataloader, device=device)
        
        valid_loss, valid_accuracy = valid_eval(model, dataloader=valid_dataloader, device=device)
        
        history["Train Loss"].append(train_loss)
        history["Valid Loss"].append(valid_loss)
        history["Train Acc"].append(train_accuracy)
        history["Valid Acc"].append(valid_accuracy)
        history["LRs"].extend(lrs)
        
        if valid_loss < min_avg_loss:
            print(f"Validation Loss Improved ({min_avg_loss} ---> {valid_loss})")
            min_avg_loss = valid_loss
            max_accuracy = valid_accuracy
            best_model_weights = copy.deepcopy(model.state_dict())
            path = f"best_model.bin"
            torch.save(model.state_dict(), path)
            print(f"Model Saved")
        
        print('-'*89)
        
    
    print("Minimum Loss: {:.4f} Best Accuracy: {:.2f}".format(min_avg_loss, max_accuracy))
    
    model.load_state_dict(best_model_weights)
    return model, history

In [44]:
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
params

2431528

In [45]:
for p in model.parameters():
    print(p.shape)

torch.Size([16, 1, 3, 3, 3])
torch.Size([16])
torch.Size([16])
torch.Size([16])
torch.Size([32, 16, 3, 3, 3])
torch.Size([32])
torch.Size([256, 9408])
torch.Size([256])
torch.Size([32, 256])
torch.Size([32])
torch.Size([8, 32])
torch.Size([8])


In [46]:
from torch.optim import lr_scheduler

In [47]:
lr = 0.01
epochs = 10

In [48]:
valid_dataset = FractureDetectDataset(df = df_valid, path = TRAIN_DATA_PATH, transform=None)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)

In [49]:
os.environ.get('PYTORCH_CUDA_ALLOC_CONF')

'max_split_size_mb:3000'

In [50]:
torch.cuda.empty_cache()

In [51]:
model_parameters = filter(lambda parameter: parameter.requires_grad, model.parameters())
optimizer = torch.optim.AdamW(model_parameters, lr=lr, weight_decay = 0.1)
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=500, eta_min=0.001)
criterion = nn.BCEWithLogitsLoss(reduction = 'none')

#test 1 epoch
model, history = train_epochs(model, optimizer, scheduler,criterion, device=device, epochs=epochs)

Epoch: 0
1.1702892780303955 :time
Training Loss over a batch: 43.0704; Training Loss over a single value: 0.6730 Accuracy: 93.75%
Validation Loss over a batch: 6626.7544; Training Loss over a single value: 138.0574 Accuracy: 78.20%
Validation Loss Improved (1000000000.0 ---> 6626.75439453125)
Model Saved
-----------------------------------------------------------------------------------------
Epoch: 1
0.8580894470214844 :time
Training Loss over a batch: 715.3229; Training Loss over a single value: 11.1769 Accuracy: 70.31%
Validation Loss over a batch: 1554.6804; Training Loss over a single value: 32.3892 Accuracy: 78.05%
Validation Loss Improved (6626.75439453125 ---> 1554.680419921875)
Model Saved
-----------------------------------------------------------------------------------------
Epoch: 2
0.8980307579040527 :time
Training Loss over a batch: 304.8258; Training Loss over a single value: 4.7629 Accuracy: 71.88%
Validation Loss over a batch: 238.6096; Training Loss over a single val

In [52]:
"""import torch
from GPUtil import showUtilization as gpu_usage
from numba import cuda

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache()"""


'import torch\nfrom GPUtil import showUtilization as gpu_usage\nfrom numba import cuda\n\ndef free_gpu_cache():\n    print("Initial GPU Usage")\n    gpu_usage()                             \n\n    torch.cuda.empty_cache()\n\n    cuda.select_device(0)\n    cuda.close()\n    cuda.select_device(0)\n\n    print("GPU Usage after emptying the cache")\n    gpu_usage()\n\nfree_gpu_cache()'

In [53]:
gpu_usage()

| ID | GPU | MEM |
------------------
|  0 |  0% |  6% |


In [54]:
gc.collect()
torch.cuda.empty_cache()
del model
gpu_usage()

| ID | GPU | MEM |
------------------
|  0 |  0% |  6% |
